In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)


# <spark.read.csv> used to load the CSV data into the DF 'df'.
# header= true: to state that the first row contains col. headers.
# inferSchema= to infer the schema from the data.
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)



In [4]:
# 2. Create a temporary view of the DataFrame.


# <.createOrReplaceTempView()> to create the temp view named 'home_sales'
df.createOrReplaceTempView("home_sales")



In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?


# Grouping by year_sold
# SQL query to calculate #3
# <AVG()> to calculate the average price for each year
# <ROUND()> to round the result to two decimal places
avg_4_bedroom_per_year = """
SELECT year(date) AS year_sold, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold
"""

# <.show()> to display the DF
spark.sql(avg_4_bedroom_per_year).show()



+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?


# SQL query to calculate #4
# <SELECT> this clause selects the avg price rounded to 2 decimal places.
# <FROM> this clause states where to extract data from.
# <WHERE> this clause filters the data based on bedrooms = 3 AND bathrooms = 3.
# <GROUP BY> this clause groups the data by the year the house was built.
# <ORDER BY> to order in ascending by the date it was built.
# <AVG()> to calculate the avg price for each year.
date_built_3_bed_bath = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# <.show()> to display the DF
spark.sql(date_built_3_bed_bath).show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?


# SQL query to calculate #5
# <SELECT> this clause selects the avg price rounded to 2 decimal places.
# <FROM> this clause states where to extract data from.
# <WHERE> clause filters the data based on 3 bed/bathrooms, 2 floors, and
### the sqft greater than  or equal to 2000 ft.
# <GROUP BY> clause groups by the date the house was built
# <ORDER BY> to order in ascending by the date it was built
date_built_3_bed_bath_2_floors_2000sqft = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

# <.show()> to display the DF
spark.sql(date_built_3_bed_bath_2_floors_2000sqft).show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.


# <time.time()> function that measures the runtime for this query.
start_time = time.time()

# SQL query to calculate #6.
# <SELECT> this clause selects the avg price rounded to 2 decimal places.
# <FROM> this clause states where to extract data from.
# <GROUP BY> clause groups by the view rating.
# <HAVING> clause filters the groups based on the price being greater than or equal to $350k.
# <ORDER BY...DESC> to order in descending by the view rating.
price_view_rating_350k = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# saving the query as 'result' variable
result = spark.sql(price_view_rating_350k)

# <.show()> to display the DF
result.show()

# print() to display the time it took to run the query
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.0026862621307373 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.


# <spark.catalog.cacheTable()> to cache the 'home_sales' temp table
spark.catalog.cacheTable("home_sales")


In [10]:
# 8. Check if the table is cached.


# <spark.catalog.isCached()> checks the 'home_sales' table to see if it is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.


# <time.time()> function that measures the runtime for this query.
start_time = time.time()

# saving the query as 'result_1' variable
result_1 = spark.sql(price_view_rating_350k)

# <.show()> to display the DF
result_1.show()

# print() to display the time it took to run the query
print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 2.2764251232147217 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data


# <.partitionBY()> function to partition the 'df' by the 'date_built' field which organizes the data.
# <.write.model('overwrite')> to write a parquet
# <.parquet()> function to save as "partitioned_home_sales.parquet"
df.write.mode('overwrite').partitionBy('date_built').parquet("partitioned_home_sales.parquet")


In [13]:
# 11. Read the parquet formatted data.


# <spark.read.parquet()> reads the parquet data from the PATH and saved as a variable 'p_df'
p_df = spark.read.parquet("partitioned_home_sales.parquet")


In [14]:
# 12. Create a temporary table for the parquet data.


# <.createOrReplaceTempView()> to create the temp view named 'parquet_home_sales'
p_df.createOrReplaceTempView("parquet_home_sales")


In [15]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.


# <time.time()> function that measures the runtime for this query.
start_time = time.time()

# saving the query as 'result_2' variable
result_2 = spark.sql(price_view_rating_350k)

# <.show()> to display the DF
result_2.show()

# print() to display the time it took to run the query
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.6900224685668945 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.


# <spark.catalog.uncacheTable()> function to uncache the 'home_sales' temp table to release memory.
spark.catalog.uncacheTable("home_sales")


In [17]:
# 15. Check if the home_sales is no longer cached


# <spark.catalog.isCached()> function to determine if 'home_sales' is no longer cached.
print(spark.catalog.isCached('home_sales'))



False
